<a href="https://colab.research.google.com/github/DLSeed/DeepLearning/blob/main/sovits_f0%E6%95%B0%E6%8D%AE%E9%9B%86%E4%B8%80%E9%94%AE%E5%88%B6%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 创建单人数据集

**[sovits合集导航](https://github.com/IceKyrin/sovits_guide)**


**格式参考vits专栏三件套（评论区）**[vits注解](https://www.bilibili.com/read/cv18478187)

95%的问题都可以参考专栏解决，剩下的我也不会了

建议下载**公开歌声数据集opencpop**，进行多人模型训练，直接转22050hz

**切片一下，一段语音超过60s可能会炸gpu**[自动切片机](https://github.com/openvpi/audio-slicer)（要按歌手分id）

本专栏使用GPU运行，cpu解析f0速度比较慢，cpu直接运行脚本可能会卡死

本专栏一次只能做一个id的数据，清数据、断联重启，重复做几次（改id）。

参考文末，合并多个数据集的压缩包即可

后续：[一键训练](https://colab.research.google.com/drive/1DexYpwWIdD_RRqQ165l-YoWMzFAHIbPy)

后续：[一键合成](https://colab.research.google.com/drive/1F3VpHCi9eridGw1F1hbqR7qhXGKuSCus#scrollTo=vjkgBV7j2cVJ)

**关联midi版（仅供语音转换）**

[sovit midi版数据集制作](https://colab.research.google.com/drive/1ynEZGX9FG8gFRZ3PX2tvNXkEq094yTmR)

数据集格式：假设名字为paimeng

/paimeng/wavs/train(val)/xxx.wav，打包成paimeng.zip上传到谷歌云盘里
（在谷歌云盘的根目录，建立dataset文件夹，然后放进去）

**你的工程文件夹叫paimeng，里面装着wavs和其子目录**
**把工程文件夹右键打包**

建个文件夹 /paimeng/wavs/val/xxx.wav
就下面这个格式，然后把**paimeng这个文件夹**右键压缩成paimeng.zip传上来
```
paimeng
└───wavs
    ├───val
    │   ├───xxx1-xxx1.wav
    │   ├───...
    │   └───Lxx-0xx8.wav
    └───train
        ├───xx2-0xxx2.wav
        ├───...
        └───xxx7-xxx007.wav
```


In [1]:
#@title 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title 数据集设置
#@markdown 数据集名称（别带zip，就是普通英文名称）
DATASETNAME = "lamianer2"  #@param {type:"string"}
#@markdown 压缩包路径（谷歌盘路径）
ZIP_PATH = "/content/drive/MyDrive/dataset/"  #@param {type:"string"}
ZIP_PATH = ZIP_PATH + DATASETNAME+".zip"
DATASETPATH = "/content/" + DATASETNAME
!cp {ZIP_PATH} {DATASETPATH}.zip
%cd /content/
!unzip -q {DATASETNAME}.zip
!pip install torchaudio soundfile
import os
import soundfile
import torchaudio
#@markdown 自动将采样率转换为22050，如需其他采样率，请自行修改代码
def resample_to_22050(audio_path):
    raw_audio, raw_sample_rate = torchaudio.load(audio_path)
    if raw_sample_rate != 22050:
      audio_22050 = torchaudio.transforms.Resample(orig_freq=raw_sample_rate, new_freq=22050)(raw_audio)[0]
      soundfile.write(audio_path, audio_22050, 22050)

for i in os.listdir(f"{DATASETPATH}/wavs/train"):
  resample_to_22050(f"{DATASETPATH}/wavs/train/{i.split('.')[0]}.wav")
for i in os.listdir(f"{DATASETPATH}/wavs/val"):
  resample_to_22050(f"{DATASETPATH}/wavs/val/{i.split('.')[0]}.wav")

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#@title 自动预处理(处理两千条数据，花3min左右，按比例估算)
!pip install torchcrepe
%cd {DATASETPATH}
!git clone https://github.com/bshall/hubert.git
%cd hubert
!python encode.py soft  {DATASETPATH}/wavs {DATASETPATH}/soft --extension .wav 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72.3 MB 5.8 kB/s 
     |████████████████████████████████| 213 kB 69.1 MB/s 
  Attempting uninstall: librosa
    Found existing installation: librosa 0.8.1
    Uninstalling librosa-0.8.1:
      Successfully uninstalled librosa-0.8.1
/content/lamianer2
Cloning into 'hubert'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 105 (delta 53), reused 80 (delta 32), pack-reused 0
Receiving objects: 100% (105/105), 469.75 KiB | 5.00 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/content/lamianer2/hubert
Loading hubert checkpoint
/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list,

In [4]:
#@title 生成txt
#@markdown 这是给多人数据集准备的，**单人填写"|"**；多人填写角色id，如"|0|"、"|1|"（编号从0开始）

#@markdown 若使用预模型加速训练，id从|10|开始，最大为15，也就是说预模型保留了6个空位，详见**一键训练**对预模型的介绍
ID = "|0|"  #@param {type:"string"}
%cd {DATASETPATH}
import os 
with open("train.txt","w") as f:
  for i in os.listdir(f"{DATASETPATH}/soft/train"):
    f.write(f"{DATASETPATH}/wavs/train/{i.split('.')[0]}.wav{ID}{DATASETPATH}/soft/train/{i}|{DATASETPATH}/pitch/train/{i.split('.')[0]}.npy\n")

with open("val.txt","w") as f:
  for i in os.listdir(f"{DATASETPATH}/soft/val"):
    f.write(f"{DATASETPATH}/wavs/val/{i.split('.')[0]}.wav{ID}{DATASETPATH}/soft/val/{i}|{DATASETPATH}/pitch/val/{i.split('.')[0]}.npy\n")

/content/lamianer2


In [5]:
#@markdown 等这个下载完
%cd /content
!git clone https://github.com/innnky/so-vits-svc
!pip install pyworld

/content
Cloning into 'so-vits-svc'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 99 (delta 25), reused 83 (delta 14), pack-reused 0
Unpacking objects: 100% (99/99), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212 kB 4.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyworld: filename=pyworld-0.3.0-cp37-cp37m-linux_x86_64.whl size=609490 sha256=00f29eb52de2e076c183608dfd8e27dd34af983ca4c8c3e5078ccc54064e26a8
  Stored in directory: /root/.cache/pip/wheels/e7/7c/11/c775fffa0e1e7b05a6604b4323408a77f80fb4ab304d96b5c6
Successfully built pyworld


In [6]:
#@title 生成pitch文件
%cd /content/so-vits-svc
!mkdir /content/{DATASETNAME}/pitch
!mkdir /content/{DATASETNAME}/pitch/train
!mkdir /content/{DATASETNAME}/pitch/val
import os
import numpy as np
from preprocess_wave import FeatureInput
featureInput = FeatureInput(22050, 256)

def resize2d(x, target_len):
    source = np.array(x)
    source[source<0.001] = np.nan
    target = np.interp(np.arange(0, len(source)*target_len, len(source))/ target_len, np.arange(0, len(source)), source)
    res = np.nan_to_num(target)
    return res

def get_end_file(dir_path, end):
    file_lists = []
    for root, dirs, files in os.walk(dir_path):
        for f_file in files:
            if f_file.endswith(end):
                file_lists.append(os.path.join(root, f_file).replace("\\", "/"))

    return file_lists


folder = "train"
wav_paths = get_end_file(f"/content/{DATASETNAME}/wavs/{folder}/", "wav")
for wav_path in wav_paths:
    soft = np.load(wav_path.replace("wavs", "soft").replace(".wav", ".npy"))
    featur_pit = featureInput.compute_f0(wav_path)
    featur_pit = resize2d(featur_pit, soft.shape[0])
    pitch = featureInput.coarse_f0(featur_pit)
    np.save(wav_path.replace("wavs", "pitch").replace(".wav", ".npy"), pitch)

folder = "val"
wav_paths = get_end_file(f"/content/{DATASETNAME}/wavs/{folder}/", "wav")
for wav_path in wav_paths:
    soft = np.load(wav_path.replace("wavs", "soft").replace(".wav", ".npy"))
    featur_pit = featureInput.compute_f0(wav_path)
    featur_pit = resize2d(featur_pit, soft.shape[0])
    pitch = featureInput.coarse_f0(featur_pit)
    np.save(wav_path.replace("wavs", "pitch").replace(".wav", ".npy"), pitch)

/content/so-vits-svc


In [7]:
#@title 打包数据集
#@markdown 处理好的压缩包路径
ZIP_PATH = "/content/drive/MyDrive/dataset/"  #@param {type:"string"}
ZIP_PATH = ZIP_PATH + "out_" + DATASETNAME + ".zip"
%cd /content/
!zip -q -r {ZIP_PATH} {DATASETNAME}
#@markdown 刷新一下谷歌盘，会多一个out_xxx.zip（根据你填的路径）

/content


多人数据集就是按序号在ID处填写"|0|"、"|1|"，每次制作单人的压缩包，然后把wavs下的文件夹按目录合并

例如：

第一个角色为paimeng，ID处写"|0|"，生成out_paimeng.zip

第二个角色为zhangsan,ID处写"|1|"，生成out_zhangsan.zip

分别解压一下：

paimeng和zhangsan的目录层次是完全相同的，把zhangsan子目录的文件，分别放到paimeng的相应文件夹，以此类推，第一个角色的文件夹，最后放的是所有文件


zhangsan的train和val.txt，把内容复制粘贴到**paimeng压缩包的相应txt内,跟在结尾一行就行，保持原格式**

**然后ctrl+h，把所有zhangsan一键替换成paimeng**

txt内**一定不能有空行**

然后右键**paimeng文件夹**，打包**zip格式**，还是生成一个out_paimeng.zip上传到云盘的dataset文件夹下

下一个脚本运行失败，自己检查是不是打包路径有问题